In [ ]:
# setup
! pip install fire hydra-core wandb -q
! git clone --depth=1 https://github.com/martin-marek/picodo.git
! python /content/picodo/download_fineweb.py 'fineweb' 26 # 2.6B tokens

In [ ]:
# imports
%cd /content/picodo
import os
from hydra import compose, initialize_config_dir
from configs import resolver_setup
from train import train_and_evaluate

In [ ]:
# training
with initialize_config_dir(f'{os.getcwd()}/configs', version_base=None):
    c = compose(config_name='base', overrides=['+model=gpt2s', '+dataset=fw_gpt2'])
c.model.activ_dtype = 'bfloat16' # reduced precision increases arithmetic intensity
c.model.H = 256 # TPU v6e has 256x256 MXU
c.opt.batch_size = 8 # smallest batch size maximizing MFU
c.opt.peak_lr = 0.002
c.opt.b2 = 0.999 # small batch size requires high b2 (arxiv.org/abs/2507.07101)
c.opt.weight_decay = 0.02 # small batch size requires small wd
c.wandb_mode = 'online' # ['disabled', 'offline', 'online']
train_and_evaluate(c)

sharding mesh: data=1, model=1
initializing model...
n_param_nonembed=84_934_656
n_param_embed=38_597_376
n_param_actual=162_129_408
getting dataset size...
reading data...
shuffling data...
splitting data...


train_loss,█▄▃▁
train_tokens_seen,▁▃▆█
n_param_actual,162129408
n_param_embed,38597376
n_param_nonembed,84934656
train_loss,7.69456
train_tokens_seen,4063232


  0%|          | 0/301591 [00:00<?, ?it/s]